In [10]:
import os
import time
import psutil
import torch
import numpy as np
import pandas as pd
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    EarlyStoppingCallback
)
from transformers.trainer_utils import get_last_checkpoint
from peft import get_peft_model, LoraConfig, TaskType
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from scipy.special import softmax

# ==============================================================================
# 1. CẤU HÌNH HỆ THỐNG
# ==============================================================================
MODEL_NAME = "tiiuae/falcon-rw-1b"
SAVE_PATH = "./results_falcon_imdb_lora_optimized" # Đổi tên folder để tránh ghi đè

if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Thiết bị: {device} - Falcon-1B_LoRA_IMDB.ipynb:31")
if torch.cuda.is_available():
    print(f"🔥 GPU: {torch.cuda.get_device_name(0)} - Falcon-1B_LoRA_IMDB.ipynb:33")

# ==============================================================================
# 2. XỬ LÝ DỮ LIỆU
# ==============================================================================
def load_data():
    print("\n Đang tải dữ liệu IMDB - Falcon-1B_LoRA_IMDB.ipynb:39")
    dataset = load_dataset("imdb")
    
    print(f"Đang tải Tokenizer: {MODEL_NAME} - Falcon-1B_LoRA_IMDB.ipynb:42")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        tokenizer.pad_token_id = tokenizer.eos_token_id
    tokenizer.padding_side = "right"

    def tokenize_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

    print("Tokenizing (Đợi xíu)... - Falcon-1B_LoRA_IMDB.ipynb:53")
    tokenized = dataset.map(tokenize_function, batched=True)
    
    tokenized = tokenized.remove_columns(["text"])
    tokenized.set_format("torch") 

    return tokenized["train"], tokenized["test"], tokenizer

# ==============================================================================
# 3. MODEL & LORA 
# ==============================================================================
def setup_model(tokenizer):
    print(f"\n Đang tải Model: {MODEL_NAME} - Falcon-1B_LoRA_IMDB.ipynb:65")
    
    is_bf16_supported = torch.cuda.is_bf16_supported()
    dtype = torch.bfloat16 if is_bf16_supported else torch.float16
    print(f"> Chế độ Precision: {dtype} - Falcon-1B_LoRA_IMDB.ipynb:69")

    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME,
        num_labels=2,
        torch_dtype=dtype,
        use_safetensors=True 
    )
    model.config.pad_token_id = tokenizer.pad_token_id
    model.config.use_cache = False 
    
    model.gradient_checkpointing_disable() 

    print("Cài đặt LoRA - Falcon-1B_LoRA_IMDB.ipynb:82")
    peft_config = LoraConfig(
        task_type=TaskType.SEQ_CLS,
        inference_mode=False,
        r=8,
        lora_alpha=32,
        lora_dropout=0.1,
        target_modules=["query_key_value", "dense"],
        bias="none"
    )
    model = get_peft_model(model, peft_config)
    
    model.print_trainable_parameters()
    model.to(device)
    return model

# ==============================================================================
# 4. METRICS
# ==============================================================================
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    if isinstance(logits, tuple): logits = logits[0]
        
    predictions = np.argmax(logits, axis=-1)
    probs = softmax(logits, axis=1)[:, 1]

    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)
    try: roc_auc = roc_auc_score(labels, probs)
    except: roc_auc = 0.0
        
    return {
        'accuracy': acc, 
        'f1': f1, 
        'precision': precision, 
        'recall': recall, 
        'roc_auc': roc_auc
    }

# ==============================================================================
# 5. MAIN
# ==============================================================================
def main():
    train_data, eval_data, tokenizer = load_data()
    model = setup_model(tokenizer)
    
    use_bf16 = torch.cuda.is_bf16_supported()
    use_fp16 = not use_bf16

    training_args = TrainingArguments(
        output_dir=SAVE_PATH,
        num_train_epochs=3,
        
        per_device_train_batch_size=16, 
        gradient_accumulation_steps=2,
        
        gradient_checkpointing=False,
        
        save_strategy="epoch",
        eval_strategy="epoch",
        save_total_limit=1,
        load_best_model_at_end=True,
        metric_for_best_model="accuracy",
        
        learning_rate=2e-4, 
        warmup_steps=100,
        logging_steps=50,
        
        bf16=use_bf16,
        fp16=use_fp16,
        
        report_to="none",
        dataloader_num_workers=0 
    )

    torch.cuda.empty_cache()

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_data,
        eval_dataset=eval_data,
        processing_class=tokenizer,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )

    # --- Train ---
    print(f"\n Bắt đầu huấn luyện TỐI ƯU (Batch=32, Checkpoint=False) - Falcon-1B_LoRA_IMDB.ipynb:170")
    last_ckpt = get_last_checkpoint(SAVE_PATH)
    start_time = time.time()
    
    if last_ckpt:
        print(f"🔄 Tìm thấy checkpoint cũ: {last_ckpt}. Tiếp tục train... - Falcon-1B_LoRA_IMDB.ipynb:175")
        trainer.train(resume_from_checkpoint=last_ckpt)
    else:
        trainer.train()
    
    train_time = time.time() - start_time

    # --- Eval ---
    print("\n Đánh giá mô hình - Falcon-1B_LoRA_IMDB.ipynb:183")
    start_pred = time.time()
    preds = trainer.predict(eval_data)
    end_pred = time.time()
    
    m = preds.metrics
    metrics = {
        "Accuracy": m.get("test_accuracy", 0),
        "Precision": m.get("test_precision", 0),
        "Recall": m.get("test_recall", 0),
        "F1-Score": m.get("test_f1", 0),
        "ROC-AUC": m.get("test_roc_auc", 0)
    }
    
    latency = ((end_pred - start_pred) / len(eval_data)) * 1000
    
    trainer.save_model(SAVE_PATH)
    adapter_path = os.path.join(SAVE_PATH, "adapter_model.safetensors")
    if not os.path.exists(adapter_path):
        adapter_path = os.path.join(SAVE_PATH, "adapter_model.bin")
    size_mb = os.path.getsize(adapter_path) / (1024**2) if os.path.exists(adapter_path) else 0
    
    vram_mb = torch.cuda.max_memory_allocated() / (1024**2) if torch.cuda.is_available() else 0

    print("\n - Falcon-1B_LoRA_IMDB.ipynb:207" + "="*50)
    print("====== REPORT: Falcon1B + LoRA (A5000 Optimized) ====== - Falcon-1B_LoRA_IMDB.ipynb:208")
    print("= - Falcon-1B_LoRA_IMDB.ipynb:209"*50)
    print("1. Classification Metrics: - Falcon-1B_LoRA_IMDB.ipynb:210")
    print(f"Accuracy:   {metrics['Accuracy']:.4f} - Falcon-1B_LoRA_IMDB.ipynb:211")
    print(f"Precision:  {metrics['Precision']:.4f} - Falcon-1B_LoRA_IMDB.ipynb:212")
    print(f"Recall:     {metrics['Recall']:.4f} - Falcon-1B_LoRA_IMDB.ipynb:213")
    print(f"F1Score:    {metrics['F1Score']:.4f} - Falcon-1B_LoRA_IMDB.ipynb:214")
    print(f"ROCAUC:     {metrics['ROCAUC']:.4f} - Falcon-1B_LoRA_IMDB.ipynb:215")
    
    print("\n2. Efficiency Metrics: - Falcon-1B_LoRA_IMDB.ipynb:217")
    print(f"Training Time:     {train_time:.2f} s - Falcon-1B_LoRA_IMDB.ipynb:218")
    print(f"Inference Latency: {latency:.4f} ms/sample - Falcon-1B_LoRA_IMDB.ipynb:219")
    print(f"Adapter Size:      {size_mb:.2f} MB - Falcon-1B_LoRA_IMDB.ipynb:220")
    print(f"Peak VRAM:         {vram_mb:.2f} MB - Falcon-1B_LoRA_IMDB.ipynb:221")
    print("= - Falcon-1B_LoRA_IMDB.ipynb:222"*50)

    df_data = {
        "Metric": list(metrics.keys()) + ["Training Time (s)", "Latency (ms)", "Size (MB)", "VRAM (MB)"],
        "Value": list(metrics.values()) + [train_time, latency, size_mb, vram_mb]
    }
    pd.DataFrame(df_data).to_csv(os.path.join(SAVE_PATH, "final_report_optimized.csv"), index=False)
    print(f"✅ Kết quả đã lưu tại: {os.path.join(SAVE_PATH, 'final_report_optimized.csv')} - Falcon-1B_LoRA_IMDB.ipynb:229")

if __name__ == "__main__":
    main()

🚀 Thiết bị: cuda - Falcon-1B_LoRA_IMDB.ipynb:31
🔥 GPU: NVIDIA RTX A5000 - Falcon-1B_LoRA_IMDB.ipynb:33

 Đang tải dữ liệu IMDB - Falcon-1B_LoRA_IMDB.ipynb:39
Đang tải Tokenizer: tiiuae/falcon-rw-1b - Falcon-1B_LoRA_IMDB.ipynb:42
Tokenizing (Đợi xíu)... - Falcon-1B_LoRA_IMDB.ipynb:53

 Đang tải Model: tiiuae/falcon-rw-1b - Falcon-1B_LoRA_IMDB.ipynb:65
> Chế độ Precision: torch.bfloat16 - Falcon-1B_LoRA_IMDB.ipynb:69


Some weights of FalconForSequenceClassification were not initialized from the model checkpoint at tiiuae/falcon-rw-1b and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Cài đặt LoRA - Falcon-1B_LoRA_IMDB.ipynb:82
trainable params: 2,363,392 || all params: 1,313,992,704 || trainable%: 0.1799


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 50256, 'bos_token_id': 50256}.



 Bắt đầu huấn luyện TỐI ƯU (Batch=32, Checkpoint=False) - Falcon-1B_LoRA_IMDB.ipynb:170


...

c:\Users\TTNT1\miniconda3\envs\nlp\lib\site-packages\torch\nn\parallel\_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
c:\Users\TTNT1\miniconda3\envs\nlp\lib\site-packages\torch\cuda\nccl.py:16: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn("PyTorch is not compiled with NCCL support")
c:\Users\TTNT1\miniconda3\envs\nlp\lib\site-packages\torch\nn\parallel\_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
c:\Users\TTNT1\miniconda3\envs\nlp\lib\site-packages\torch\cuda\nccl.py:16: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn("PyTorch is not compiled with NCCL support")



 Đánh giá mô hình - Falcon-1B_LoRA_IMDB.ipynb:183


c:\Users\TTNT1\miniconda3\envs\nlp\lib\site-packages\torch\nn\parallel\_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(



 - Falcon-1B_LoRA_IMDB.ipynb:207==================================================
====== REPORT: Falcon1B + LoRA (A5000 Optimized) ====== - Falcon-1B_LoRA_IMDB.ipynb:208
= - Falcon-1B_LoRA_IMDB.ipynb:209= - Falcon-1B_LoRA_IMDB.ipynb:209= - Falcon-1B_LoRA_IMDB.ipynb:209= - Falcon-1B_LoRA_IMDB.ipynb:209= - Falcon-1B_LoRA_IMDB.ipynb:209= - Falcon-1B_LoRA_IMDB.ipynb:209= - Falcon-1B_LoRA_IMDB.ipynb:209= - Falcon-1B_LoRA_IMDB.ipynb:209= - Falcon-1B_LoRA_IMDB.ipynb:209= - Falcon-1B_LoRA_IMDB.ipynb:209= - Falcon-1B_LoRA_IMDB.ipynb:209= - Falcon-1B_LoRA_IMDB.ipynb:209= - Falcon-1B_LoRA_IMDB.ipynb:209= - Falcon-1B_LoRA_IMDB.ipynb:209= - Falcon-1B_LoRA_IMDB.ipynb:209= - Falcon-1B_LoRA_IMDB.ipynb:209= - Falcon-1B_LoRA_IMDB.ipynb:209= - Falcon-1B_LoRA_IMDB.ipynb:209= - Falcon-1B_LoRA_IMDB.ipynb:209= - Falcon-1B_LoRA_IMDB.ipynb:209= - Falcon-1B_LoRA_IMDB.ipynb:209= - Falcon-1B_LoRA_IMDB.ipynb:209= - Falcon-1B_LoRA_IMDB.ipynb:209= - Falcon-1B_LoRA_IMDB.ipynb:209= - Falcon-1B_LoRA_IMDB.ipynb:209= -

KeyError: 'F1Score'

In [2]:

import time
import os
import torch
import psutil
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
from peft import PeftModel
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from scipy.special import softmax
from datasets import load_dataset

# === BƯỚC 1: FIX LỖI BẢO MẬT & ĐƯỜNG DẪN ===
import transformers.utils.hub
os.environ["TRANSFORMERS_VERIFY_SCHEDULED_CVES"] = "False"
# Bypass kiểm tra phiên bản Torch cho CVE-2025-32434
transformers.utils.hub.is_torch_greater_or_equal_than_2_6 = lambda: True

MODEL_NAME = "tiiuae/falcon-rw-1b"
# Đảm bảo đường dẫn này khớp với thư mục LoRA của bạn
SAVE_PATH = "./results_falcon_imdb_lora_optimized" 

# === BƯỚC 2: KHỞI TẠO LẠI (Tránh lỗi NameError) ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("📥 Đang chuẩn bị Tokenizer và Dữ liệu Test... - Falcon-1B_LoRA_IMDB.ipynb:27")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "right"

dataset_test = load_dataset("imdb")["test"]

def tokenize_fn(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tokenized_test = dataset_test.map(tokenize_fn, batched=True)
tokenized_test.set_format("torch")

print("🏗️ Đang tải Model và LoRA Adapter... - Falcon-1B_LoRA_IMDB.ipynb:41")
dtype = torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16
base_model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, 
    num_labels=2, 
    torch_dtype=dtype,
    trust_remote_code=True,
    use_safetensors=True
)
base_model.config.pad_token_id = tokenizer.pad_token_id

# Load trọng số LoRA
model = PeftModel.from_pretrained(base_model, SAVE_PATH)
model.to(device)
model.eval()

# Khởi tạo Trainer với DataCollator để tránh lỗi Padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
trainer = Trainer(
    model=model,
    args=TrainingArguments(output_dir="./temp", per_device_eval_batch_size=8, report_to="none"),
    data_collator=data_collator
)

# === BƯỚC 3: CHẠY EVALUATION ===
print("🚀 Đang đánh giá 25,000 mẫu tập Test (Vui lòng đợi)... - Falcon-1B_LoRA_IMDB.ipynb:66")
start_pred = time.time()
predictions_output = trainer.predict(tokenized_test)
end_pred = time.time()

# === BƯỚC 4: TÍNH TOÁN ĐỦ 10 CHỈ SỐ ===
# --- 1-5: Classification Metrics ---
raw_logits = predictions_output.predictions
if isinstance(raw_logits, tuple): raw_logits = raw_logits[0]
true_labels = predictions_output.label_ids
pred_labels = np.argmax(raw_logits, axis=-1)
pred_probs = softmax(raw_logits, axis=1)[:, 1]

acc = accuracy_score(true_labels, pred_labels)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, pred_labels, average='binary')
roc_auc = roc_auc_score(true_labels, pred_probs)

# --- 6: Training Time (Nếu mất session, hãy điền thủ công giá trị bạn đã train) ---
try:
    final_training_time = train_time 
except NameError:
    final_training_time = 0.0 # Placeholder

# --- 7: Inference Latency ---
latency = ((end_pred - start_pred) / len(tokenized_test)) * 1000 # ms/sample

# --- 8: Adapter Size ---
model_size_mb = 0
for f in ['adapter_model.safetensors', 'adapter_model.bin']:
    path = os.path.join(SAVE_PATH, f)
    if os.path.exists(path):
        model_size_mb = os.path.getsize(path) / (1024**2)
        break

# --- 9-10: Resource Usage ---
process = psutil.Process(os.getpid())
ram_usage = process.memory_info().rss / (1024 ** 2)
vram_usage = torch.cuda.max_memory_allocated() / (1024 ** 2) if torch.cuda.is_available() else 0

# === BƯỚC 5: IN BÁO CÁO CHI TIẾT ===
print("\n - Falcon-1B_LoRA_IMDB.ipynb:106" + "="*60)
print("📊 FINAL REPORT: Falcon1B + LoRA (IMDB)  Full 10 Metrics - Falcon-1B_LoRA_IMDB.ipynb:107")
print("= - Falcon-1B_LoRA_IMDB.ipynb:108"*60)
print(f"1. Classification Metrics: - Falcon-1B_LoRA_IMDB.ipynb:109")
print(f"Accuracy:          {acc:.4f} - Falcon-1B_LoRA_IMDB.ipynb:110")
print(f"Precision:         {precision:.4f} - Falcon-1B_LoRA_IMDB.ipynb:111")
print(f"Recall:            {recall:.4f} - Falcon-1B_LoRA_IMDB.ipynb:112")
print(f"F1Score:          {f1:.4f} - Falcon-1B_LoRA_IMDB.ipynb:113")
print(f"ROCAUC:           {roc_auc:.4f} - Falcon-1B_LoRA_IMDB.ipynb:114")

print(f"\n2. Efficiency Metrics: - Falcon-1B_LoRA_IMDB.ipynb:116")
print(f"Training Time:      {final_training_time:.2f} s - Falcon-1B_LoRA_IMDB.ipynb:117")
print(f"Inference Latency:  {latency:.4f} ms/sample - Falcon-1B_LoRA_IMDB.ipynb:118")
print(f"Adapter Size (Disk): {model_size_mb:.2f} MB - Falcon-1B_LoRA_IMDB.ipynb:119")
print(f"Peak RAM Usage:     {ram_usage:.2f} MB - Falcon-1B_LoRA_IMDB.ipynb:120")
print(f"Peak VRAM Usage:    {vram_usage:.2f} MB - Falcon-1B_LoRA_IMDB.ipynb:121")
print("= - Falcon-1B_LoRA_IMDB.ipynb:122"*60)

# === BƯỚC 6: LƯU CSV ===
results_df = pd.DataFrame({
    "Metric": ["Accuracy", "Precision", "Recall", "F1", "ROC-AUC",
               "Training Time (s)", "Inference Latency (ms)", "Adapter Size (MB)",
               "Peak RAM (MB)", "Peak VRAM (MB)"],
    "Value": [acc, precision, recall, f1, roc_auc, 
              final_training_time, latency, model_size_mb, 
              ram_usage, vram_usage]
})

results_file = os.path.join(SAVE_PATH, 'falcon_imdb_lora_full_report.csv')
results_df.to_csv(results_file, index=False)
print(f"✅ Báo cáo đầy đủ (10 chỉ số) đã lưu tại: {results_file} - Falcon-1B_LoRA_IMDB.ipynb:136")

📥 Đang chuẩn bị Tokenizer và Dữ liệu Test... - Falcon-1B_LoRA_IMDB.ipynb:27


Map: 100%|██████████| 25000/25000 [00:09<00:00, 2687.19 examples/s]


🏗️ Đang tải Model và LoRA Adapter... - Falcon-1B_LoRA_IMDB.ipynb:41


`torch_dtype` is deprecated! Use `dtype` instead!
Some weights of FalconForSequenceClassification were not initialized from the model checkpoint at tiiuae/falcon-rw-1b and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🚀 Đang đánh giá 25,000 mẫu tập Test (Vui lòng đợi)... - Falcon-1B_LoRA_IMDB.ipynb:66


c:\Users\TTNT1\miniconda3\envs\nlp\lib\site-packages\torch\nn\parallel\_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(



 - Falcon-1B_LoRA_IMDB.ipynb:106============================================================
📊 FINAL REPORT: Falcon1B + LoRA (IMDB)  Full 10 Metrics - Falcon-1B_LoRA_IMDB.ipynb:107
= - Falcon-1B_LoRA_IMDB.ipynb:108= - Falcon-1B_LoRA_IMDB.ipynb:108= - Falcon-1B_LoRA_IMDB.ipynb:108= - Falcon-1B_LoRA_IMDB.ipynb:108= - Falcon-1B_LoRA_IMDB.ipynb:108= - Falcon-1B_LoRA_IMDB.ipynb:108= - Falcon-1B_LoRA_IMDB.ipynb:108= - Falcon-1B_LoRA_IMDB.ipynb:108= - Falcon-1B_LoRA_IMDB.ipynb:108= - Falcon-1B_LoRA_IMDB.ipynb:108= - Falcon-1B_LoRA_IMDB.ipynb:108= - Falcon-1B_LoRA_IMDB.ipynb:108= - Falcon-1B_LoRA_IMDB.ipynb:108= - Falcon-1B_LoRA_IMDB.ipynb:108= - Falcon-1B_LoRA_IMDB.ipynb:108= - Falcon-1B_LoRA_IMDB.ipynb:108= - Falcon-1B_LoRA_IMDB.ipynb:108= - Falcon-1B_LoRA_IMDB.ipynb:108= - Falcon-1B_LoRA_IMDB.ipynb:108= - Falcon-1B_LoRA_IMDB.ipynb:108= - Falcon-1B_LoRA_IMDB.ipynb:108= - Falcon-1B_LoRA_IMDB.ipynb:108= - Falcon-1B_LoRA_IMDB.ipynb:108= - Falcon-1B_LoRA_IMDB.ipynb:108= - Falcon-1B_LoRA_IMDB.ip